In [8]:
pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [9]:
import numpy as np
import pandas as pd
import torch
import gc; gc.enable()
from sklearn.impute import KNNImputer
from sklearn.metrics import roc_auc_score
import category_encoders as ce
from sklearn.linear_model import LogisticRegression, HuberRegressor
import warnings; warnings.filterwarnings("ignore")

AFTER_PROCESS_TRAIN_PATH = "./after_train.csv"
AFTER_PROCESS_TEST_PATH = "./after_test.csv"
SAMPLE_SUBMISSION_PATH = "./sample_submission.csv"

SUBMISSION_PATH = "./final_0811619.csv"

In [10]:
train_df = pd.read_csv(AFTER_PROCESS_TRAIN_PATH)
test_df = pd.read_csv(AFTER_PROCESS_TEST_PATH)
submission = pd.read_csv(SAMPLE_SUBMISSION_PATH)

In [11]:
folds_dict = {f'Fold 1': [['A', 'B', 'C'], ['D', 'E']],
               'Fold 2': [['A', 'B', 'D'], ['C', 'E']],
               'Fold 3': [['A', 'B', 'E'], ['C', 'D']],
               'Fold 4': [['A', 'C', 'D'], ['B', 'E']],
               'Fold 5': [['A', 'C', 'E'], ['B', 'D']],
               'Fold 6': [['A', 'D', 'E'], ['B', 'C']],
               'Fold 7': [['B', 'C', 'D'], ['A', 'E']],
               'Fold 8': [['B', 'C', 'E'], ['A', 'D']],
               'Fold 9': [['B', 'D', 'E'], ['A', 'C']],
               'Fold 10': [['C', 'D', 'E'], ['A', 'B']] 
               }

In [12]:
features = ['loading', 'm1', 'm2', 'measurement_17', 'measurement_0', 'measurement_1', 'measurement_2', 
            'measurement_median', 'measurement_max', 'measurement_min', 'measurement_skew']

In [13]:
test_predictions = np.zeros((test_df.shape[0], 1))

auc_score = []

for fold in folds_dict.keys():

    model = torch.load(f'model{fold}.pt')
    test_pred = model.predict_proba(test_df[features].values)[:, 1].reshape(-1, 1)
    test_predictions += test_pred * 0.1

In [14]:
submission['failure'] = test_predictions
submission.to_csv(SUBMISSION_PATH, index=False)